<a href="https://colab.research.google.com/github/keinam53/Machine_learning/blob/main/Uczenie_nienadzorowane/2_Redukcja_wymiarowosci/3_PCA_wine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###PCA - wine
1. [Import bibliotek](#0)
2. [Załadowanie danych](#1)
3. [Podział na zbiór treningowy i testowy](#2)
4. [Standaryzacja](#3)
5. [PCA](#4)

### <a name='0'></a> Import bibliotek

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

np.set_printoptions(precision=4, suppress=True, edgeitems=5, linewidth=200)

### <a name='1'></a> Załadowanie danych

In [2]:
df_raw = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)
df = df_raw.copy()
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [3]:
data = df.iloc[:,1:]
data.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [4]:
target = df[0]
target.head()

0    1
1    1
2    1
3    1
4    1
Name: 0, dtype: int64

In [5]:
target.value_counts()

2    71
1    59
3    48
Name: 0, dtype: int64

### <a name='2'></a> Podział na zbiór treningowy i testowy

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target)

print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')

X_train shape: (133, 13)
X_test shape: (45, 13)


### <a name='3'></a> Standaryzacja

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)
print(X_train_std[:3])
print(f'\n{X_test_std[:3]}')

[[ 1.7223 -0.588   1.1472  1.5602 -0.1125  0.8975 -0.7291  1.33    2.0082  3.6969 -1.7675 -0.9776 -0.197 ]
 [ 0.5663  1.3338 -0.8891 -0.209  -0.6444  0.316   0.6854 -0.7205 -0.1659 -0.2261 -0.2709  0.5146  1.0894]
 [ 0.7528 -0.5435  0.3046  0.2761  1.0844  1.1634  0.7887 -1.2725  1.5627  0.6602  0.0372  0.5576  1.3809]]

[[ 1.424  -0.161  -0.2571 -0.4372  0.353   1.1468  1.3462 -1.1148  1.438   0.4352  1.2257  0.7585  2.7187]
 [-1.0868 -0.4546 -0.1869 -0.2946 -1.2429 -1.0964 -0.5329  1.2512  0.1193 -1.0719  0.4774 -0.5328 -0.8144]
 [ 0.6533 -0.5969  0.9366  0.8468 -0.7109  0.5652 -0.9459  1.2512  1.2776  3.1346 -1.7675 -1.2359 -0.3342]]


### <a name='4'></a> PCA

In [8]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)
print(X_train_pca.shape)
print(f'\n{X_train_pca.shape}')

(133, 3)

(133, 3)


Wyjaśniona wariancja

In [9]:
results = pd.DataFrame(data={'explained_variance_ratio': pca.explained_variance_ratio_})
results['cumulative'] = results['explained_variance_ratio'].cumsum()
results['component'] = results.index + 1
results

,explained_variance_ratio,cumulative,component
0,0.348609,0.348609,1
1,0.190268,0.538877,2
2,0.118539,0.657417,3


In [10]:
fig = go.Figure(data=[go.Bar(x=results['component'], y=results['explained_variance_ratio'], name='explained variance ratio'),
                      go.Scatter(x=results['component'], y=results['cumulative'], name='cumulative explained variance')],
                layout=go.Layout(title=f'PCA - {pca.n_components_} components', width=950, template='plotly_dark'))
fig.show()

In [11]:
X_train_pca_df = pd.DataFrame(data=np.c_[X_train_pca, y_train], columns=['pca_1', 'pca_2', 'pca_3', 'target'])
X_train_pca_df.head()

,pca_1,pca_2,pca_3,target
0,1.022040,-3.712922,1.259138,3.0
1,-0.680071,-0.491079,-0.847649,1.0
2,-2.559340,-1.505751,0.446233,1.0
3,0.033604,1.896970,0.582661,2.0
4,0.352699,1.952504,-0.545413,2.0


In [13]:
px.scatter_3d(X_train_pca_df, 'pca_1', 'pca_2', 'pca_3', color='target', template='plotly_dark', width=950)

In [14]:
X_train_pca[:5]

array([[ 1.022 , -3.7129,  1.2591],
       [-0.6801, -0.4911, -0.8476],
       [-2.5593, -1.5058,  0.4462],
       [ 0.0336,  1.897 ,  0.5827],
       [ 0.3527,  1.9525, -0.5454]])

In [15]:
X_test_pca[:5]

array([[-3.5207, -1.5603, -0.554 ],
       [ 1.3444,  2.0431, -0.1343],
       [ 1.5984, -2.6268,  0.6492],
       [-1.9727, -0.4051,  0.339 ],
       [-0.1314,  1.0292,  0.6498]])